这是一个不是很完善的版本，如果你有什么意见，可以联系我。

有部分内容参考了https://py-xdh.readthedocs.io/zh-cn/latest/index.html 的内容。



# PySCF简介
PySCF（Python-based Simulations of Chemistry Framework）是一个基于Python的开源量子化学软件包，专注于分子、晶体和自定义哈密顿量的电子结构计算。

PySCF的优势：适合教育、研究和快速开发，社区活跃（GitHub开源），支持混合编程。

PySCF的局限：大规模计算性能可能不如传统Fortran软件（如Gaussian、VASP），但对中小体系足够高效。

# 0. 环境准备
首先需要一个Python环境，如果读者还没有Python环境的话，推荐使用Anaconda或者miniconda。通过conda管理Python环境十分方便。

有了Python环境以后，安装Pyscf

pip 安装
```
pip install pyscf
```
conda 安装
```
conda install conda-forge::pyscf
```

# 1. 分子结构
在这一小节中，这里我们讨论 PySCF 下的分子结构、基组与电子积分的调用过程。

## 1.1 分子的构建

In [1]:
from pyscf import gto
mol = gto.Mole()
mol.atom = [("H",(0,0,0)),("H",(0,0,1))]
mol.basis = "6-31G"
mol.build()

在上面的例子中，我们创建了一个最简单的STO-3G基组的氢分子。

`mol = gto.Mole()` 是初始化一个 gto.Mole 类到变量 mol 中。该类保存了与分子和基组有关的绝大部分信息、以及电子积分的调用方式。

`mol.atom = ...` 通过较为简单的方式作分子的结构定义。默认情况下，长度的默认单位是 Angstrom。

`mol.basis = "6-31G"` 是定义分子使用的基组。

`mol.build()` 对分子进行构建。

`gto.Mole`类的一些其他可以调整的参数：

`mol.spin` 指定分子的自旋数，也就是自旋向上电子数和自旋向下电子数的差值。默认值时 0。

`mol.charge` 指定分子的带电数，默认值是0。

## 1.2 分子的一些性质输出

原子核坐标

In [2]:
coords = mol.atom_coords()
coords

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.88972612]])

原子核电荷

In [3]:
charges = mol.atom_charges()
charges

array([1, 1], dtype=int32)

有了原子核电荷和原子核坐标，我们就可以计算原子核排斥能了
$$\sum_{A=1}^M \sum_{B>A}^M \frac{Z_A Z_B}{R_{A B}}$$

PySCF中可以直接调用`mol.energy_nuc()`获取核排斥能

In [4]:
mol.energy_nuc()

0.52917721092

## 1.3 电子积分
让我们回顾一下采取了Born-Oppenheimer近似后的电子哈密顿量：
$$
H_{e l e c}=-\sum_{i=1}^N \frac{1}{2} \nabla_i^2-\sum_{i=1}^N \sum_{A=1}^M \frac{Z_A}{r_{i A}}+\sum_{i=1}^N \sum_{j>i}^N \frac{1}{r_{i j}}
$$
我们要在给定的基组上计算这些算符在给定基函数的矩阵元，将上述算符转化为矩阵，从而在给定的有限基组下近似得求解该哈密顿量的基态

方便起见，我们将上面的电子哈密顿量表达为：
$$
H_{elec} = H_{T} + H_{V} + H_{ee} 
$$
其中动能项
$$
H_{T} = -\sum_{i=1}^N \frac{1}{2} \nabla_i^2
$$
势能项
$$
H_{V} = -\sum_{i=1}^N \sum_{A=1}^M \frac{Z_A}{r_{i A}}
$$
以及电子排斥项：
$$
H_{ee} = \sum_{i=1}^N \sum_{j>i}^N \frac{1}{r_{i j}}
$$

首先我们可以计算基组的重叠积分：
$$
\left\langle i \mid j\right\rangle=S_{i j} 
$$
(Modern Quantum Chemistry, Eq. 3.34)

In [5]:
S = mol.intor("int1e_ovlp")
S

array([[1.        , 0.65829205, 0.25330671, 0.41171988],
       [0.65829205, 1.        , 0.41171988, 0.74978653],
       [0.25330671, 0.41171988, 1.        , 0.65829205],
       [0.41171988, 0.74978653, 0.65829205, 1.        ]])

由S矩阵可见我们的基组各基函数之间并不正交。

我们也可以计算动能项在给定基组下的矩阵元：
$$
\langle i | H_{T} | j \rangle
$$


In [6]:
T = mol.intor("int1e_kin")
T

array([[1.39567828, 0.25973508, 0.02815422, 0.11133304],
       [0.25973508, 0.2419167 , 0.11133304, 0.14656382],
       [0.02815422, 0.11133304, 1.39567828, 0.25973508],
       [0.11133304, 0.14656382, 0.25973508, 0.2419167 ]])

我们也可以计算势能项在给定基组下的矩阵元：
$$
\langle i | H_{V} | j \rangle
$$


In [7]:
V = mol.intor('int1e_nuc')
V

array([[-2.18153331, -1.085489  , -0.49705041, -0.70046637],
       [-1.085489  , -1.10173122, -0.70046637, -0.87620977],
       [-0.49705041, -0.70046637, -2.18153331, -1.085489  ],
       [-0.70046637, -0.87620977, -1.085489  , -1.10173122]])

计算电子排斥项在基组下的矩阵元
$$
\langle \ ij| H_{ee} | kl \rangle
$$
在书中也将这个积分简写为
$$
\langle ij | kl \rangle
$$

In [8]:
eri = mol.intor("int2e")
eri.shape

(4, 4, 4, 4)

当然，可以计算的电子积分还有很多很多，详见PySCF的文档。

在这里我们不细究这些电子积分的具体计算过程是怎么样做的。我们也不细究各个基组是怎么定义的，尽管这是量子化学中非常重要的一个研究方向。

# 2. Hartree-Fock 自洽场计算

## 2.1 PySCF的Hartree Fock 计算拆解

In [9]:
from pyscf import scf
mf = scf.RHF(mol)
mf.run()

converged SCF energy = -1.09480796286051


以上就是一个简单的RHF计算，在PySCF中已经是一个封装得很好得方法了，非常简单就可以调用。接下来我们尝试对PySCF中的RHF方法进行拆解。

我们要求解的方程是Hartree-Fock方程：
$$
\left[h(1)+\sum_{b=1}^N\left(J_b(1)-K_b(1)\right)\right] \chi_a(1)=\sum_{b=1}^N \varepsilon_{b a} \chi_b(1)
$$
(Modern Quantum Chemistry, Eq. 3.48)

其中$h(1)$表示的是单电子部分，也就是
$$
h(1)=-\frac{1}{2} \nabla_1^2-\sum_A \frac{Z_A}{r_{1 A}}
$$
(Modern Quantum Chemistry, Eq. 3.5)

可以用`mf.get_hcore()`来调用$h(1)$在基组下的矩阵元

In [10]:
h1 = mf.get_hcore()
h1

array([[-0.78585503, -0.82575392, -0.46889618, -0.58913333],
       [-0.82575392, -0.85981452, -0.58913333, -0.72964595],
       [-0.46889618, -0.58913333, -0.78585503, -0.82575392],
       [-0.58913333, -0.72964595, -0.82575392, -0.85981452]])

$h(1)$就是我们之前计算过的动能项和势能项之和，可以验证它们是相等的。

In [11]:
import numpy as np
np.isclose(T+V , h1).all()

True

`mf.make_rdm1()`方法可以得到单粒子密度矩阵

In [13]:
dm = mf.make_rdm1()
dm

NPArrayWithTag([[0.1651197 , 0.19063096, 0.1651197 , 0.19063096],
                [0.19063096, 0.22008375, 0.19063096, 0.22008375],
                [0.1651197 , 0.19063096, 0.1651197 , 0.19063096],
                [0.19063096, 0.22008375, 0.19063096, 0.22008375]])

所谓的密度矩阵就是描述电子在基函数上分布情况的物理量

接下来我们就要计算Hartree Fock方法中最重要的部分，库伦算符$J$和交换算符$K$在基函数上的矩阵元

根据Modern Quantum Chemistry Eq. 3.154
$$
\begin{aligned}
F_{\mu \nu} & =H_{\mu \nu}^{\mathrm{core}}+\sum_a^{N / 2} \sum_{\lambda \sigma} C_{\lambda a} C_{\sigma a}^*[2(\mu v \mid \sigma \lambda)-(\mu \lambda \mid \sigma v)] \\
& =H_{\mu \nu}^{\mathrm{core}}+\sum_{\lambda \sigma} P_{\lambda \sigma}\left[(\mu \nu \mid \sigma \lambda)-\frac{1}{2}(\mu \lambda \mid \sigma v)\right] \\
& =H_{\mu \nu}^{\mathrm{core}}+G_{\mu \nu}
\end{aligned}
$$

In [16]:
J = np.einsum('ijkl,kl-> ij',eri, dm)
J

array([[1.31017042, 0.78260456, 0.34843549, 0.50985313],
       [0.78260456, 0.94283295, 0.50985313, 0.74362043],
       [0.34843549, 0.50985313, 1.31017042, 0.78260456],
       [0.50985313, 0.74362043, 0.78260456, 0.94283295]])

这部分可能比较难，需要结合书本和代码仔细学习一下。

我们也可以调用`mf.get_j()`使用PySCF提供的方法计算$J$ 不难验证和我们计算的$J$是一致的。

In [17]:
mf.get_j(dm=dm)

array([[1.31017042, 0.78260456, 0.34843549, 0.50985313],
       [0.78260456, 0.94283295, 0.50985313, 0.74362043],
       [0.34843549, 0.50985313, 1.31017042, 0.78260456],
       [0.50985313, 0.74362043, 0.78260456, 0.94283295]])

In [20]:
np.isclose(mf.get_j(dm=dm),J).all()

True

用同样的方法也可以计算$K$

In [21]:
K = np.einsum('ikjl,kl-> ij',eri, dm)
K

array([[0.87626561, 0.77458172, 0.59324087, 0.6816691 ],
       [0.77458172, 0.7923071 , 0.6816691 , 0.75227283],
       [0.59324087, 0.6816691 , 0.87626561, 0.77458172],
       [0.6816691 , 0.75227283, 0.77458172, 0.7923071 ]])

In [24]:
np.isclose(mf.get_k(dm = dm),K).all()

True

根据Eq. 3.154:
$$
\begin{aligned}
F_{\mu \nu} & =H_{\mu \nu}^{\mathrm{core}}+\sum_a^{N / 2} \sum_{\lambda \sigma} C_{\lambda a} C_{\sigma a}^*[2(\mu v \mid \sigma \lambda)-(\mu \lambda \mid \sigma v)] \\
& =H_{\mu \nu}^{\mathrm{core}}+\sum_{\lambda \sigma} P_{\lambda \sigma}\left[(\mu \nu \mid \sigma \lambda)-\frac{1}{2}(\mu \lambda \mid \sigma v)\right] \\
& =H_{\mu \nu}^{\mathrm{core}}+G_{\mu \nu}
\end{aligned}
$$

In [47]:
Fock = h1+J-0.5*K
Fock

array([[ 0.08618259, -0.43044022, -0.41708113, -0.42011475],
       [-0.43044022, -0.31313512, -0.42011475, -0.36216194],
       [-0.41708113, -0.42011475,  0.08618259, -0.43044022],
       [-0.42011475, -0.36216194, -0.43044022, -0.31313512]])

也可以用`mf.get_fock()`方法直接得到Fock矩阵。

In [33]:
np.isclose(mf.get_fock(dm = dm),Fock).all()

True

至此我们已经完成了Hartree-Fock计算中所有需要的组件！我们可以写一个我们自己的小型Hartree-Fock代码！

## 2.2 一个小型的Hartree-Fock SCF程序
我们按照Modern Quantum Chemistry P146 （汉化版P127-128）的流程来构建一个小型的Hartree Fock算法。

1．确定一个分子（一组核坐标 $\left\{\mathbf{R}_A\right\}$ ，原子序数 $Z_A$ ，电子数 $N$ ）和基组 $\left\{\phi_A\right\}$ ．

In [49]:
from pyscf import gto, scf
import numpy as np
import scipy
def geometry_h2o(bl = 1.0 , theta = np.pi *104/180):
    """get geometry for h2o"""
    geometry = []
    geometry.append(("O",(0,0,0)))
    geometry.append(("H",(0,bl*np.sin(theta/2),bl*np.cos(theta/2))))
    geometry.append(("H",(0,-bl*np.sin(theta/2),bl*np.cos(theta/2))))
    return geometry

mol = gto.Mole()
mol.atom = geometry_h2o()
mol.basis = "sto-3g"
mol.build()

n_atom = mol.natm    # 原子数
n_mo = n_ao = mol.nao  # 分子轨道数，原子轨道数（基函数个数）
n_occ = mol.nelectron//2  # 占据轨道数

2．计算所需的分子积分，$S_{\mu \nu}, H_{\mu \nu}^{\text {core }},(\mu \nu \mid \lambda \sigma)$ ．

In [50]:
S = mol.intor("int1e_ovlp")
H1 = mol.intor("int1e_kin") + mol.intor("int1e_nuc")
eri = mol.intor("int2e")

3．对角化重叠矩阵 $\mathbf{S}$ ，用（3．167）或（3．169）得到变换矩阵 X．
$$
\mathbf{X}=\mathbf{S}^{-1 / 2}=\mathbf{U} \mathbf{s}^{-1 / 2} \mathbf{U}^{\dagger}
$$

In [51]:
X = scipy.linalg.fractional_matrix_power(S, -0.5)
# X = np.linalg.inv(np.linalg.cholesky(S).T)

4．猜测一个密度矩阵 $\mathbf{P}$ ．

In [74]:
D = scf.get_init_guess(mol) # PySCF提供了猜测初始密度矩阵的方法
D = np.random.random((n_ao, n_ao))  # 也可以随机一个密度矩阵。
D = np.zeros((n_ao, n_ao))    # 初始化为0也不是不行
# 当然，在算大型体系的时候还是根据一定的方法找一个好的初猜。
D

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

5．用密度矩阵 $\mathbf{P}$ 计算（3．154）中的矩阵 $\mathbf{G}$ 和双电子积分 $(\mu v \mid \lambda \sigma)$ ．
$$
F_{\mu \nu} =H_{\mu \nu}^{\mathrm{core}}+\sum_{\lambda \sigma} P_{\lambda \sigma}\left[(\mu \nu \mid \sigma \lambda)-\frac{1}{2}(\mu \lambda \mid \sigma v)\right] 
$$
6． $\mathbf{G}$ 加核 Hamiltonian 矩阵得到 Fock 矩阵 $\mathbf{F}=\mathbf{H}^{\text {core }}+\mathbf{G}$ ．

7．计算变换后的 Fock 矩阵 $\mathbf{F}^{\prime}=\mathbf{X}^{\dagger} \mathbf{F X}$ ．

8．对角化 $\mathbf{F}^{\prime}$ 得到 $\mathbf{C}^{\prime}, \boldsymbol{\varepsilon}$ ．

9．根据（3．14）用 $\mathbf{C}$ 构建新密度矩阵 $\mathbf{P}$ ．

In [75]:
D_old = np.random.random((n_ao, n_ao))
count = 0

while (not np.allclose(D, D_old)):
    if count > 500:
        raise ValueError("SCF not converged!")
    count += 1
    D_old = D
    F = H1 + np.einsum("uvkl, kl -> uv", eri, D) - 0.5 * np.einsum("ukvl, kl -> uv", eri, D) # 计算G (J-1/2 K)，与H1相加得到Fock矩阵
    Fp = X.T @ F @ X                     # 根据S的正交化结果计算在正交基组下的 Fock 矩阵 F^\prime
    e, Cp = np.linalg.eigh(Fp)          # 对角化F^\prime 得到Fock矩阵的本征值和本征向量
    C = X @ Cp                          # 将 C 变换回非正交基组下。这里的C就是分子轨道和原子轨道之间的变换矩阵
    D = 2 * C[:, :n_occ] @ C[:, :n_occ].T

10．确定该过程是否收敛，即确定（10）中密度矩阵是否与前一个密度矩阵在某种判据下相同．若未收玫，回到（5）用新密度矩阵计算．

11．若收玫，则用得到的解表示出 $\mathbf{C}, \mathbf{P}, \mathbf{F}$ 等，即计算期望值和其他想求的量．

In [78]:
E_elec = np.einsum("ij,ij ->",H1,D) + 0.5 * np.einsum("uvkl, uv, kl ->", eri, D, D) - 0.25 * np.einsum("ukvl, uv, kl ->", eri, D, D)

别忘了把原子核能加上

In [79]:
E_nuc = mol.energy_nuc()

In [80]:
print(f"SCF converges in {count} iterations")
print(f"Electron energy = {E_elec} Hartree")
print(f"Nuclear energy = {E_nuc} Hartree")
print(f"Total energy = {E_elec + E_nuc} Hartree")

SCF converges in 18 iterations
Electron energy = -83.76746223064198 Hartree
Nuclear energy = 8.802603134549392 Hartree
Total energy = -74.9648590960926 Hartree


和PySCF计算得到的对比一下

In [82]:
mf = scf.RHF(mol).run()
print(f"RHF energy with PySCF: {mf.e_tot} Hartree")
print(f"Error = {E_elec + E_nuc - mf.e_tot} Hartree")

converged SCF energy = -74.9648590960925
RHF energy with PySCF: -74.96485909609247 Hartree
Error = -1.2789769243681803e-13 Hartree


至此一个完整的SCF流程就完成了